# FaceGreeter Dependency Test

This notebook tests dependencies (`flask-cors`, `python-dotenv`, `combined_app.py`) to identify the cause of kernel crashes in `face_greeter_camera.ipynb` (April 22, kernel `d318a6a9-3644-46c2-9b04-51dde4632ce7`). It uses the stable threading setup from `minimal_face_greeter_ngrok.ipynb` (April 22) and clones `https://github.com/ATEMMETA/ai-greeter-core`. The goal is to isolate the crashing dependency and stabilize the Flask server for `https://facegreeter.vercel.app`.

## Prerequisites
- Google Colab environment with internet access.
- `NGROK_AUTHTOKEN` in Colab Secrets.
- GitHub repository: `https://github.com/ATEMMETA/ai-greeter-core`.

## Steps
1. Install minimal dependencies (`flask`, `pyngrok`) and test Flask.
2. Add `python-dotenv` and `.env` setup.
3. Add `flask-cors` and clone repository.
4. Run `combined_app.py` with threading.
5. Test `/hello` endpoint.

**Goal**: Identify the crashing dependency and stabilize the Flask API.

In [ ]:
# Test 1: Minimal Flask + NGROK
import os
import psutil
import threading
import time
from flask import Flask, jsonify
from pyngrok import ngrok
from google.colab import userdata

# Log initial resource usage
print(f'Initial RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Initial CPU: {psutil.cpu_percent()}%')

# Check NGROK_AUTHTOKEN
ngrok_token = userdata.get('NGROK_AUTHTOKEN')
if not ngrok_token:
    raise ValueError('NGROK_AUTHTOKEN not found in Colab Secrets.')

# Purge pip cache
!pip cache purge

# Install minimal dependencies
!pip install --no-cache-dir flask==3.0.3 pyngrok==7.1.6

# Verify installed packages
!pip list | grep -E 'flask|pyngrok'

# Log resource usage after install
print(f'Post-install RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Post-install CPU: {psutil.cpu_percent()}%')

# Set ngrok auth token
ngrok.set_auth_token(ngrok_token)

# Create Flask app
app = Flask(__name__)

@app.route('/hello', methods=['GET'])
def hello():
    return jsonify({"success": True, "message": "Test 1: Minimal Flask"})

# Run Flask in a thread
def run_flask():
    print('Starting Flask server on http://0.0.0.0:8000')
    app.run(host='0.0.0.0', port=8000, debug=False, use_reloader=False)

print('Starting Flask server...')
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

# Wait for Flask to start
time.sleep(5)

# Start ngrok tunnel
try:
    public_url = ngrok.connect(8000).public_url
    print(f'Flask API available at: {public_url}')
except Exception as e:
    print(f'Ngrok failed to start: {str(e)}')
    os.system('pkill python')
    os.system('pkill ngrok')
    os.system('fuser -k 8000/tcp')
    raise

# Check processes and port
os.system('ps aux | grep python')
os.system('netstat -tuln | grep 8000')

# Log resource usage after Flask
print(f'Post-Flask RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Post-Flask CPU: {psutil.cpu_percent()}%')

print(f'Test 1 complete. Test /hello at {public_url}/hello.')
print('Press Ctrl+C to stop, then proceed to the next cell.')
try:
    flask_thread.join()
except KeyboardInterrupt:
    print('Stopping Flask server...')
    os.system('pkill python')
    os.system('pkill ngrok')
    os.system('fuser -k 8000/tcp')

In [ ]:
# Test 2: Add python-dotenv and .env
import os
import psutil
import threading
import time
from flask import Flask, jsonify
from pyngrok import ngrok
from google.colab import userdata
from dotenv import load_dotenv

# Log initial resource usage
print(f'Initial RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Initial CPU: {psutil.cpu_percent()}%')

# Install python-dotenv
!pip install --no-cache-dir python-dotenv==1.0.1

# Verify installed packages
!pip list | grep -E 'flask|pyngrok|python-dotenv'

# Create .env file
ngrok_token = userdata.get('NGROK_AUTHTOKEN')
if not ngrok_token:
    raise ValueError('NGROK_AUTHTOKEN not found in Colab Secrets.')
env_path = '/content/.env'
with open(env_path, 'w') as f:
    f.write(f'NGROK_AUTHTOKEN={ngrok_token}\n')
    f.write('VERCEL_URL=https://facegreeter.vercel.app\n')

# Load environment variables
load_dotenv(env_path)
ngrok_token = os.getenv('NGROK_AUTHTOKEN')
if not ngrok_token:
    raise ValueError('NGROK_AUTHTOKEN not found in .env file.')

# Set ngrok auth token
ngrok.set_auth_token(ngrok_token)

# Create Flask app
app = Flask(__name__)

@app.route('/hello', methods=['GET'])
def hello():
    return jsonify({"success": True, "message": "Test 2: With python-dotenv"})

# Run Flask in a thread
def run_flask():
    print('Starting Flask server on http://0.0.0.0:8000')
    app.run(host='0.0.0.0', port=8000, debug=False, use_reloader=False)

print('Starting Flask server...')
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

# Wait for Flask to start
time.sleep(5)

# Start ngrok tunnel
try:
    public_url = ngrok.connect(8000).public_url
    print(f'Flask API available at: {public_url}')
except Exception as e:
    print(f'Ngrok failed to start: {str(e)}')
    os.system('pkill python')
    os.system('pkill ngrok')
    os.system('fuser -k 8000/tcp')
    raise

# Check processes and port
os.system('ps aux | grep python')
os.system('netstat -tuln | grep 8000')

# Log resource usage after Flask
print(f'Post-Flask RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Post-Flask CPU: {psutil.cpu_percent()}%')

print(f'Test 2 complete. Test /hello at {public_url}/hello.')
print('Press Ctrl+C to stop, then proceed to the next cell.')
try:
    flask_thread.join()
except KeyboardInterrupt:
    print('Stopping Flask server...')
    os.system('pkill python')
    os.system('pkill ngrok')
    os.system('fuser -k 8000/tcp')

In [ ]:
# Test 3: Add flask-cors and clone repository
import os
import psutil
import threading
import time
import shutil
import subprocess
from flask import Flask, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from google.colab import userdata
from dotenv import load_dotenv

# Log initial resource usage
print(f'Initial RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Initial CPU: {psutil.cpu_percent()}%')

# Install flask-cors
!pip install --no-cache-dir flask-cors==5.0.0

# Verify installed packages
!pip list | grep -E 'flask|pyngrok|python-dotenv'

# Clone repository
if os.path.exists('/content/ai-greeter-core'):
    shutil.rmtree('/content/ai-greeter-core')
os.makedirs('/content/ai-greeter-core', exist_ok=True)
repo_url = 'https://github.com/ATEMMETA/ai-greeter-core.git'
clone_command = f'git clone {repo_url} /content/ai-greeter-core'
print(f'Cloning repository: {clone_command}')
clone_process = subprocess.run(clone_command, shell=True, capture_output=True, text=True)
required_files = ['/content/ai-greeter-core/requirements.txt', '/content/ai-greeter-core/combined_app.py']
files_missing = [f for f in required_files if not os.path.exists(f)]
if clone_process.returncode != 0 or files_missing:
    print(f'Git clone failed or files missing. Error: {clone_process.stderr}')
    print(f'Missing files: {files_missing}')
    raise FileNotFoundError('Required files missing after git clone.')
else:
    print('Git clone succeeded. Files found:', required_files)

# Load .env
env_path = '/content/.env'
load_dotenv(env_path)
ngrok_token = os.getenv('NGROK_AUTHTOKEN')
if not ngrok_token:
    raise ValueError('NGROK_AUTHTOKEN not found in .env file.')

# Set ngrok auth token
ngrok.set_auth_token(ngrok_token)

# Create Flask app with CORS
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "https://facegreeter.vercel.app"}})

@app.route('/hello', methods=['GET'])
def hello():
    return jsonify({"success": True, "message": "Test 3: With flask-cors and clone"})

# Run Flask in a thread
def run_flask():
    print('Starting Flask server on http://0.0.0.0:8000')
    app.run(host='0.0.0.0', port=8000, debug=False, use_reloader=False)

print('Starting Flask server...')
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

# Wait for Flask to start
time.sleep(5)

# Start ngrok tunnel
try:
    public_url = ngrok.connect(8000).public_url
    print(f'Flask API available at: {public_url}')
except Exception as e:
    print(f'Ngrok failed to start: {str(e)}')
    os.system('pkill python')
    os.system('pkill ngrok')
    os.system('fuser -k 8000/tcp')
    raise

# Check processes and port
os.system('ps aux | grep python')
os.system('netstat -tuln | grep 8000')

# Log resource usage after Flask
print(f'Post-Flask RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Post-Flask CPU: {psutil.cpu_percent()}%')

print(f'Test 3 complete. Test /hello at {public_url}/hello.')
print('Press Ctrl+C to stop, then proceed to the next cell.')
try:
    flask_thread.join()
except KeyboardInterrupt:
    print('Stopping Flask server...')
    os.system('pkill python')
    os.system('pkill ngrok')
    os.system('fuser -k 8000/tcp')

In [ ]:
# Test 4: Run combined_app.py
import os
import psutil
import threading
import time
from pyngrok import ngrok
from google.colab import userdata
from dotenv import load_dotenv

# Log initial resource usage
print(f'Initial RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Initial CPU: {psutil.cpu_percent()}%')

# Verify combined_app.py exists
if not os.path.exists('/content/ai-greeter-core/combined_app.py'):
    raise FileNotFoundError('combined_app.py not found. Ensure git clone succeeded.')

# Load .env
env_path = '/content/.env'
load_dotenv(env_path)
ngrok_token = os.getenv('NGROK_AUTHTOKEN')
if not ngrok_token:
    raise ValueError('NGROK_AUTHTOKEN not found in .env file.')

# Set ngrok auth token
ngrok.set_auth_token(ngrok_token)

# Run Flask from combined_app.py in a thread
def run_flask():
    print('Starting Flask server on http://0.0.0.0:8000')
    os.system('python /content/ai-greeter-core/combined_app.py')

print('Starting Flask server...')
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

# Wait for Flask to start
time.sleep(5)

# Start ngrok tunnel
try:
    public_url = ngrok.connect(8000).public_url
    print(f'Flask API available at: {public_url}')
except Exception as e:
    print(f'Ngrok failed to start: {str(e)}')
    os.system('pkill python')
    os.system('pkill ngrok')
    os.system('fuser -k 8000/tcp')
    raise

# Check processes and port
os.system('ps aux | grep python')
os.system('netstat -tuln | grep 8000')

# Log resource usage after Flask
print(f'Post-Flask RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Post-Flask CPU: {psutil.cpu_percent()}%')

print(f'Test 4 complete. Test /hello at {public_url}/hello.')
print('Press Ctrl+C to stop.')
try:
    flask_thread.join()
except KeyboardInterrupt:
    print('Stopping Flask server...')
    os.system('pkill python')
    os.system('pkill ngrok')
    os.system('fuser -k 8000/tcp')